# Import Dependencies

In [12]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Loading Data

## Mounting Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


## Unzipping files

In [ ]:
import zipfile

zip_file_path = "/content/drive/MyDrive/Personal/Personal Projects/NLP Product Recommender/data.csv.zip"
zip_file_path ="data.csv.zip"
extract_to = "/content/"


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
  zip_ref.extractall(extract_to)

## Load CSV

In [4]:
df = pd.read_csv("/content/data.csv")
df.head()

,id,slug,title,imgs,brand,category,vendor,used,address,availability,...,discounted_price,specifications,description,delivery_fee,delivery_details,warranty,warranty_type,average_rating,num_ratings,reviews
0,0,https://www.mega.pk/mobiles_products/23522/Not...,Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...,['https://www.mega.pk/items_images/Nothing+Pho...,NaN,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,1,https://www.mega.pk/mobiles_products/23458/Opp...,Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...,['https://www.mega.pk/items_images/Oppo+F21+Pr...,OPPO,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,"{'RAM': '8gb', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,https://www.mega.pk/mobiles_products/24393/Tec...,Tecno Spark 10,['https://www.mega.pk/items_images/Tecno+Spark...,Tecno,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '4GB,8GB', 'Memory quantity': '', 'Int...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[]
3,3,https://www.mega.pk/mobiles_products/24259/Viv...,Vivo V27 5G,['https://www.mega.pk/items_images/Vivo+V27+5G...,Vivo,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '8GB,12GB', 'Memory quantity': '', 'In...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[]
4,4,https://www.mega.pk/mobiles_products/24204/App...,Apple Iphone 15 Pro Max,['https://www.mega.pk/items_images/Apple+Iphon...,Apple,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


# Cleaning Text

In [5]:
  def clean_row(row):
    title = row['title'] if pd.notna(row['title']) else ''
    brand = row['brand'] if pd.notna(row['brand']) else ''
    category = row['category'] if pd.notna(row['category']) else ''
    specs = ' '.join([f"{k}:{v}"for k,v in eval(row['specifications']).items()]) if pd.notna(row['specifications']) else ''
    return f"{title} {brand} {category} {specs}"

df['cleaned text'] = df.apply(clean_row, axis=1)
df = df.dropna(subset=['cleaned text']) # drops the text without any descriptions
df.head()


,id,slug,title,imgs,brand,category,vendor,used,address,availability,...,specifications,description,delivery_fee,delivery_details,warranty,warranty_type,average_rating,num_ratings,reviews,cleaned text
0,0,https://www.mega.pk/mobiles_products/23522/Not...,Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...,['https://www.mega.pk/items_images/Nothing+Pho...,NaN,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...
1,1,https://www.mega.pk/mobiles_products/23458/Opp...,Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...,['https://www.mega.pk/items_images/Oppo+F21+Pr...,OPPO,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,"{'RAM': '8gb', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...
2,2,https://www.mega.pk/mobiles_products/24393/Tec...,Tecno Spark 10,['https://www.mega.pk/items_images/Tecno+Spark...,Tecno,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,"{'RAM': '4GB,8GB', 'Memory quantity': '', 'Int...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[],"Tecno Spark 10 Tecno Mobile RAM:4GB,8GB Memor..."
3,3,https://www.mega.pk/mobiles_products/24259/Viv...,Vivo V27 5G,['https://www.mega.pk/items_images/Vivo+V27+5G...,Vivo,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,"{'RAM': '8GB,12GB', 'Memory quantity': '', 'In...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[],"Vivo V27 5G Vivo Mobile RAM:8GB,12GB Memory q..."
4,4,https://www.mega.pk/mobiles_products/24204/App...,Apple Iphone 15 Pro Max,['https://www.mega.pk/items_images/Apple+Iphon...,Apple,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Apple Iphone 15 Pro Max Apple Mobile RAM:8GB ...


In [6]:
df.shape

(1666, 23)

# Create Embeddings

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

df['embeddings'] = df['cleaned text'].apply(lambda x: model.encode(x))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
df['embeddings']

,embeddings
0,"[-0.04479972, 0.022082357, 0.063624695, -0.073..."
1,"[-0.030580945, 0.026759638, 0.028625073, -0.03..."
2,"[-0.062415145, -0.039247576, -0.021569327, -0...."
3,"[-0.010082798, -0.04356674, 0.018539075, -0.09..."
4,"[0.0076884553, -0.0025473945, 0.035240218, -0...."
...,...
1661,"[-0.05713506, -0.004154949, 0.013521167, -0.00..."
1662,"[-0.02666272, -0.04191961, 0.01279035, -0.0212..."
1663,"[-0.060478058, 0.0045030066, 0.04209017, -0.03..."
1664,"[-0.04776419, 0.026038729, -0.0047479626, -0.0..."


# Load Embeddings

In [10]:
df.to_pickle('embeddings.pkl')

df = pd.read_pickle('embeddings.pkl') # will be used later on in integration.

,id,slug,title,imgs,brand,category,vendor,used,address,availability,...,description,delivery_fee,delivery_details,warranty,warranty_type,average_rating,num_ratings,reviews,cleaned text,embeddings
0,0,https://www.mega.pk/mobiles_products/23522/Not...,Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...,['https://www.mega.pk/items_images/Nothing+Pho...,NaN,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...,"[-0.04479972, 0.022082357, 0.063624695, -0.073..."
1,1,https://www.mega.pk/mobiles_products/23458/Opp...,Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...,['https://www.mega.pk/items_images/Oppo+F21+Pr...,OPPO,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...,"[-0.030580945, 0.026759638, 0.028625073, -0.03..."
2,2,https://www.mega.pk/mobiles_products/24393/Tec...,Tecno Spark 10,['https://www.mega.pk/items_images/Tecno+Spark...,Tecno,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,NaN,NaN,1 year,NaN,NaN,NaN,[],"Tecno Spark 10 Tecno Mobile RAM:4GB,8GB Memor...","[-0.062415145, -0.039247576, -0.021569327, -0...."
3,3,https://www.mega.pk/mobiles_products/24259/Viv...,Vivo V27 5G,['https://www.mega.pk/items_images/Vivo+V27+5G...,Vivo,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,NaN,NaN,1 year,NaN,NaN,NaN,[],"Vivo V27 5G Vivo Mobile RAM:8GB,12GB Memory q...","[-0.010082798, -0.04356674, 0.018539075, -0.09..."
4,4,https://www.mega.pk/mobiles_products/24204/App...,Apple Iphone 15 Pro Max,['https://www.mega.pk/items_images/Apple+Iphon...,Apple,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Apple Iphone 15 Pro Max Apple Mobile RAM:8GB ...,"[0.0076884553, -0.0025473945, 0.035240218, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1661,https://priceoye.pk/laptops/apple/apple-macboo...,Apple MacBook Air 13.3 Inches M1 Chip (MGN63),['https://images.priceoye.pk/apple-macbook-air...,NaN,Laptop,PriceOye,0,NaN,Not Available,...,NaN,NaN,NaN,NaN,NaN,5.0,1.0,[],Apple MacBook Air 13.3 Inches M1 Chip (MGN63) ...,"[-0.05713506, -0.004154949, 0.013521167, -0.00..."
1662,1662,https://priceoye.pk/laptops/apple/apple-macboo...,Apple MacBook Pro (MKGP3),['https://images.priceoye.pk/apple-macbook-pro...,NaN,Laptop,PriceOye,0,NaN,Not Available,...,NaN,NaN,NaN,NaN,NaN,5.0,1.0,[],Apple MacBook Pro (MKGP3) Laptop Dimensions:0...,"[-0.02666272, -0.04191961, 0.01279035, -0.0212..."
1663,1663,https://priceoye.pk/laptops/hp/hp-chromebook-1...,HP Chromebook 14 G5,['https://images.priceoye.pk/hp-chromebook-14-...,NaN,Laptop,PriceOye,0,NaN,Not Available,...,NaN,NaN,NaN,NaN,NaN,5.0,1.0,[],HP Chromebook 14 G5 Laptop Dimensions:337 x 2...,"[-0.060478058, 0.0045030066, 0.04209017, -0.03..."
1664,1664,https://priceoye.pk/laptops/dell/dell-inspiron...,Dell Inspiron 3515 15.6 inches AMD Ryzen 5 345...,['https://images.priceoye.pk/dell-inspiron-351...,NaN,Laptop,PriceOye,0,NaN,Not Available,...,NaN,NaN,NaN,NaN,NaN,5.0,1.0,[],Dell Inspiron 3515 15.6 inches AMD Ryzen 5 345...,"[-0.04776419, 0.026038729, -0.0047479626, -0.0..."


# Recommendation System

In [ ]:
def recommend_products(query, top_k=5):
  query_embedding = model.encode(query)
  df['similarity'] = df['embeddings'].apply(lambda x: cosine_similarity([query_embedding], [x]).flatten()[0])
  recommendations = df.sort_values(by='similarity',ascending=False).head(top_k)
  return recommendations[['title', 'brand', 'category', 'imgs', 'similarity']]

In [ ]:
query = "Apple Iphone 15 Pro Max"
recommendations = recommend_products(query)
recommendations

,title,brand,category,imgs,similarity
1456,Apple iPhone 14 Pro Max,NaN,Mobile,['https://images.priceoye.pk/apple-iphone-14-p...,0.621412
4,Apple Iphone 15 Pro Max,Apple,Mobile,['https://www.mega.pk/items_images/Apple+Iphon...,0.608661
57,Apple iPhone 14 Pro Max 1TB Singal Sim Non PTA,Apple,Mobile,['https://www.mega.pk/items_images/Apple+iPhon...,0.588144
121,Apple iPhone 14 Pro Max,Apple,Mobile,['https://www.mega.pk/items_images/Apple+iPhon...,0.587406
85,Apple iPhone 14 Pro Max 1TB Storage PTA Approv...,Apple,Mobile,['https://www.mega.pk/items_images/Apple+iPhon...,0.574524
